In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
quest_path = '../../data/raw/WESAD/S2/S2_quest.csv'

In [3]:
df_quest = pd.read_csv(quest_path)

In [4]:
df_quest

,# Subj;S2;;;;;;;;;;;;;;;;;;;;;;;;;
0,# ORDER;Base;TSST;Medi 1;Fun;Medi 2;sRead;fRea...
1,# START;7.08;39.55;70.19;93.38;54.42;89.51;;;;...
2,# END;26.32;50.3;77.1;87.47;100.15;56.07;91.15...
3,;;;;;;;;;;;;;;;;;;;;;;;;;;
4,# PANAS;1;1;3;2;1;3;1;1;1;2;2;2;2;1;4;3;4;4;2;...
5,# PANAS;3;2;4;1;3;3;1;2;1;4;2;4;3;1;5;4;4;4;2;...
6,# PANAS;1;1;2;3;1;2;1;1;1;1;1;1;3;1;2;1;2;3;1;...
7,# PANAS;1;1;2;3;1;1;1;1;1;1;1;1;2;1;4;1;1;3;1;...
8,# PANAS;1;1;1;2;1;1;1;1;1;1;1;1;2;1;2;1;1;2;1;...
9,;;;;;;;;;;;;;;;;;;;;;;;;;;


In [5]:
physio_path = '../../data/processed/WESAD/feature_extracted/S2.csv'

In [6]:
df_physio = pd.read_csv(physio_path)

In [7]:
df_physio

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject
0,45.711333,25.829333,29.226667,63.847120,5.413990,1.429672,0.146341,-0.001787,21.396913,1.270066,99.736129,645.380435,214.707185,170.288801,73.333333,47.615157,1,2
1,54.270000,25.437333,20.178667,63.340664,1.026754,1.209851,0.093624,-0.000175,19.393566,2.157177,105.071172,617.187500,173.499336,180.196291,70.212766,10.486861,1,2
2,50.980000,21.947333,23.119333,63.409226,2.780055,1.015014,0.109211,-0.003112,22.845275,3.592279,85.831596,749.198718,235.973170,183.345695,65.789474,5.789592,1,2
3,46.682667,15.696667,24.106667,63.432355,4.567968,0.762664,0.049744,-0.001423,20.922586,2.468841,90.106695,713.033537,146.845908,177.105020,65.000000,2.765433,1,2
4,53.356667,16.646000,22.310000,63.403083,3.464344,0.640051,0.042624,-0.001226,23.547401,3.927255,85.552438,754.407051,141.898474,176.668610,50.000000,18.231489,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,62.008667,0.096667,11.999333,63.159777,0.194601,1.083680,0.035084,0.000161,24.775107,2.215447,98.842094,642.323370,236.804537,152.719220,84.444444,0.529102,2,2
57,61.998000,0.000667,11.999333,63.148544,0.184591,1.087609,0.025663,-0.000324,21.795537,1.553626,98.327254,671.164773,320.500778,216.838097,86.046512,7.833447,2,2
58,61.994667,-0.051333,11.999333,63.145677,0.208440,1.133577,0.019665,-0.000368,23.753213,1.805575,82.246750,767.628205,239.070294,169.109433,86.842105,5.964856,2,2
59,61.979333,-0.500000,12.046000,63.143400,0.176237,1.164148,0.042966,0.000315,25.773459,2.338470,99.146434,654.861111,275.015738,193.560117,77.272727,11.237952,2,2


In [8]:
from tqdm import tqdm
import glob

In [9]:
all_file = glob.glob('../../data/processed/WESAD/feature_extracted/S*.csv')
df_all = pd.concat([pd.read_csv(file) for file in all_file], ignore_index=True, axis=0)
df_all.head

<bound method NDFrame.head of       ACC_x_mean  ACC_y_mean  ACC_z_mean  net_acc_mean  net_acc_std  EDA_mean  \
0      37.715333  -35.027333   -9.694000     62.840001     5.214880  0.630601   
1      39.252000  -47.664667    7.822000     62.549339     0.544205  0.563131   
2      39.628667  -46.246667    9.764000     62.469299     1.120992  0.523506   
3      38.927333  -48.439333    6.083333     62.442314     0.428267  0.520051   
4      38.919333  -47.006667   -0.914667     62.576766     1.166077  0.493109   
...          ...         ...         ...           ...          ...       ...   
1070   46.964000   -8.760000   -7.498667     63.226677     2.985640  1.290620   
1071   33.698667    3.725333   49.853333     62.980308     2.770939  1.290599   
1072   31.114000    4.294000   52.504667     62.972961     2.406421  1.332025   
1073   28.826667  -17.737333   17.747333     63.635677     6.112821  1.373313   
1074   42.781250   39.937500   19.921875     65.943343    13.060815  1.377572  

In [10]:
df_all['focus_label'] = -1 # new label

In [11]:
df_all

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject,focus_label
0,37.715333,-35.027333,-9.694000,62.840001,5.214880,0.630601,0.026678,-0.000766,17.748666,1.755023,75.081294,847.426471,226.949936,186.465391,93.939394,1.475494,1,8,-1
1,39.252000,-47.664667,7.822000,62.549339,0.544205,0.563131,0.014406,-0.000412,21.083279,1.630067,63.966173,958.669355,213.058396,133.561891,90.000000,5.933398,1,8,-1
2,39.628667,-46.246667,9.764000,62.469299,1.120992,0.523506,0.009367,-0.000265,21.327375,1.591487,66.372243,911.132812,134.469912,80.095987,74.193548,44.207315,1,8,-1
3,38.927333,-48.439333,6.083333,62.442314,0.428267,0.520051,0.018623,0.000048,22.168906,1.352590,78.588847,803.631757,159.088795,155.426060,58.333333,707.930928,1,8,-1
4,38.919333,-47.006667,-0.914667,62.576766,1.166077,0.493109,0.007353,-0.000203,24.011434,1.809602,65.088073,940.020161,199.098718,134.709634,63.333333,6.149627,1,8,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,46.964000,-8.760000,-7.498667,63.226677,2.985640,1.290620,0.053588,0.000966,23.230088,1.451065,81.961468,774.259868,238.947514,170.587935,70.270270,4.036088,2,15,-1
1071,33.698667,3.725333,49.853333,62.980308,2.770939,1.290599,0.020151,-0.000114,25.608455,2.211387,101.350382,641.304348,252.891267,181.866120,88.888889,1.921900,2,15,-1
1072,31.114000,4.294000,52.504667,62.972961,2.406421,1.332025,0.031676,0.000495,24.847170,1.932891,90.927874,699.776786,207.303412,159.144898,68.292683,1.107406,2,15,-1
1073,28.826667,-17.737333,17.747333,63.635677,6.112821,1.373313,0.023374,0.000278,24.011434,1.701185,98.456919,655.539773,282.197210,195.836075,74.418605,15.788516,2,15,-1


In [12]:
def quest_load(subject_id):
    subject_path = f'../../data/raw/WESAD/S{subject_id}/S{subject_id}_quest.csv'

    with open(subject_path, 'r') as f:
        lines = f.readlines()

    order_line = [l for l in lines if l.startswith('# ORDER')][0]
    order = order_line.strip().split(';')[1:-1]
    order = [o.strip().lower() for o in order if o.strip() not in ['bread','fread','sread']]

    stai_dict = {}
    valence_dict = {}
    arousal_dict = {}

    stai_lines = [l for l in lines if l.startswith('# STAI')]
    dim_lines = [l for l in lines if l.startswith('# DIM')]

    stai_score = np.array([[int(i) for i in l.strip().split(';')[1:-1] if i.strip().isdigit()] for l in stai_lines])
    dim_score = np.array([[int(i) for i in l.strip().split(';')[1:-1] if i.strip().isdigit()] for l in dim_lines])

    # DIM score
    valence_score = dim_score[:, 0]
    arousal_score = dim_score[:, 1]

    min_len = min(len(order), len(valence_score))

    valence_score = valence_score[:min_len]
    arousal_score = arousal_score[:min_len]

    # STAI score
    for i, task in enumerate(order[:min_len]):
        if task.startswith('medi'):
            task_key = 'meditation'
        else:
            task_key = task

        positives = (stai_score[:, 0] + stai_score[:, 3] + stai_score[:, 5]) / 3
        negatives = (stai_score[:, 1] + stai_score[:, 2] + stai_score[:, 4]) / 3

        scored_stai = negatives[i] - positives[i]

        valence_dict[task_key] = valence_score[i]
        arousal_dict[task_key] = arousal_score[i]
        stai_dict[task_key] = scored_stai

    return{
        'valence': valence_dict,
        'arousal': arousal_dict,
        'stai': stai_dict
    }

In [13]:
quest_dict = {}
for s in df_all['subject'].unique():
    quest_dict[s] = quest_load(s)

In [14]:
label_to_task = {
    1: 'base',
    2: 'tsst',
    3: 'fun',
    4: 'meditation'
}

In [15]:
def refined_label(lbl, dim_arousal, dim_valence, stai_score):
    if lbl in [1, 4]:
        if (dim_arousal <= 3) and (dim_valence >= 5):
            return 1
        else:
            return 0
        
    elif lbl == 2:
        if stai_score <= 0:
            if dim_arousal >= 5:
                return 1
            else:
                return 0
        else:
            return 0
        
    else:
        return 0

In [16]:
quest_dict

{np.int64(8): {'valence': {'base': np.int64(7),
   'fun': np.int64(7),
   'meditation': np.int64(6),
   'tsst': np.int64(5)},
  'arousal': {'base': np.int64(3),
   'fun': np.int64(3),
   'meditation': np.int64(4),
   'tsst': np.int64(7)},
  'stai': {'base': np.float64(-0.6666666666666667),
   'fun': np.float64(-0.6666666666666667),
   'meditation': np.float64(-1.0000000000000002),
   'tsst': np.float64(0.9999999999999998)}},
 np.int64(9): {'valence': {'base': np.int64(8),
   'tsst': np.int64(7),
   'meditation': np.int64(9),
   'fun': np.int64(7)},
  'arousal': {'base': np.int64(3),
   'tsst': np.int64(4),
   'meditation': np.int64(1),
   'fun': np.int64(3)},
  'stai': {'base': np.float64(-2.0),
   'tsst': np.float64(-0.33333333333333326),
   'meditation': np.float64(-2.6666666666666665),
   'fun': np.float64(-1.6666666666666665)}},
 np.int64(4): {'valence': {'base': np.int64(7),
   'fun': np.int64(8),
   'meditation': np.int64(5),
   'tsst': np.int64(5)},
  'arousal': {'base': np.int6

In [17]:
for i, row in tqdm(df_all.iterrows(), total=len(df_all)):
    subj = int(row['subject'])
    lbl = int(row['label'])

    if lbl not in label_to_task:
        continue

    task_name = label_to_task[lbl]
    quest = quest_dict[subj]

    # ดึงคะแนนที่ต้องใช้
    dim_arousal = quest['arousal'][task_name]
    dim_valence = quest['valence'][task_name]
    stai_score = quest['stai'][task_name] if lbl == 2 else None

    # คำนวณ label
    focus = refined_label(lbl, dim_arousal, dim_valence, stai_score)

    # บันทึกลง dataframe
    df_all.at[i, 'focus_label'] = focus

100%|██████████| 1075/1075 [00:00<00:00, 47542.93it/s]


In [18]:
df_all

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject,focus_label
0,37.715333,-35.027333,-9.694000,62.840001,5.214880,0.630601,0.026678,-0.000766,17.748666,1.755023,75.081294,847.426471,226.949936,186.465391,93.939394,1.475494,1,8,1
1,39.252000,-47.664667,7.822000,62.549339,0.544205,0.563131,0.014406,-0.000412,21.083279,1.630067,63.966173,958.669355,213.058396,133.561891,90.000000,5.933398,1,8,1
2,39.628667,-46.246667,9.764000,62.469299,1.120992,0.523506,0.009367,-0.000265,21.327375,1.591487,66.372243,911.132812,134.469912,80.095987,74.193548,44.207315,1,8,1
3,38.927333,-48.439333,6.083333,62.442314,0.428267,0.520051,0.018623,0.000048,22.168906,1.352590,78.588847,803.631757,159.088795,155.426060,58.333333,707.930928,1,8,1
4,38.919333,-47.006667,-0.914667,62.576766,1.166077,0.493109,0.007353,-0.000203,24.011434,1.809602,65.088073,940.020161,199.098718,134.709634,63.333333,6.149627,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,46.964000,-8.760000,-7.498667,63.226677,2.985640,1.290620,0.053588,0.000966,23.230088,1.451065,81.961468,774.259868,238.947514,170.587935,70.270270,4.036088,2,15,0
1071,33.698667,3.725333,49.853333,62.980308,2.770939,1.290599,0.020151,-0.000114,25.608455,2.211387,101.350382,641.304348,252.891267,181.866120,88.888889,1.921900,2,15,0
1072,31.114000,4.294000,52.504667,62.972961,2.406421,1.332025,0.031676,0.000495,24.847170,1.932891,90.927874,699.776786,207.303412,159.144898,68.292683,1.107406,2,15,0
1073,28.826667,-17.737333,17.747333,63.635677,6.112821,1.373313,0.023374,0.000278,24.011434,1.701185,98.456919,655.539773,282.197210,195.836075,74.418605,15.788516,2,15,0


In [19]:
df_all.value_counts('focus_label')

focus_label
1    595
0    480
Name: count, dtype: int64